In [1]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
os.environ['_JAVA_OPTIONS'] = f'-Djava.io.tmpdir={localfs_path}'

import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '10G'}, # I don't know what should be here
    default_reference='GRCh38'
) 

Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4138335/
Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4138335/


23/07/30 07:18:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://ac0105:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.113-cf32652c5077
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230730-0717-0.2.113-cf32652c5077.log


In [3]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [4]:
def run_pca(mtx, mtx_path, suffix):
    mtx = hl.variant_qc(mtx)
    mtx = mtx.checkpoint(localfs_path+'variant_qced_'+suffix+mtx_path)
    mtx = hl.read_matrix_table(localfs_path+'variant_qced_'+suffix+mtx_path)
    for_pca = mtx.filter_rows(mtx.variant_qc.AF[1] > 0.05)
    pruned_variant_table = hl.ld_prune(for_pca.GT, r2=0.2, bp_window_size=500000)
    for_pca = for_pca.filter_rows(hl.is_defined(pruned_variant_table[for_pca.row_key]))

    for_pca = for_pca.checkpoint(localfs_path+'for_pca_20_'+suffix+mtx_path)
    for_pca = hl.read_matrix_table(localfs_path+'for_pca_20_'+suffix+mtx_path)
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT, k=20)
    
    mtx = mtx.annotate_cols(pruned_scores = pcs[mtx.s].scores)
    mtx = mtx.checkpoint(localfs_path+'after_pca_20_'+suffix+mtx_path)
    
    return(mtx)

def run_pca_no_filter(mtx, mtx_path, suffix):
    
    for_pca = mtx.sample_rows(0.1)

    for_pca = for_pca.checkpoint(localfs_path+'subset_'+suffix+mtx_path)
    for_pca = hl.read_matrix_table(localfs_path+'subset_'+suffix+mtx_path)
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT, k=20)
    
    mtx = mtx.annotate_cols(scores_no_filter = pcs[mtx.s].scores)
    mtx = mtx.checkpoint(localfs_path+'after_pca_no_filters_'+suffix+mtx_path)
    
    return(mtx)

def remove_pca_outliers(mtx, field, last_score, mtx_path, suffix):
    
    mtx = mtx.annotate_globals(
            st = mtx.aggregate_cols(
                hl.agg.array_agg(
                    lambda pc: hl.agg.stats(pc),
                    mtx[field])
            )
        )

    mtx = mtx.annotate_cols(
            pc_outliers=hl.map(
                lambda s, st: hl.int((s > st['mean'] + (10 * st['stdev'])) | (s < st['mean'] - (10 * st['stdev']))),
                mtx[field][0:last_score],
                mtx.st
            )
        )

    mtx = mtx.filter_cols(
        hl.sum(mtx.pc_outliers) ==  0
    )
    
    mtx = mtx.checkpoint(localfs_path+'no_outliers'+mtx_path+suffix+'.mt')
    
    return(mtx)

### select matrices for skat:
1. sportsmen vs GTS 40
2. polish zeros vs GTS 40

In [5]:
mt_1 = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/gts_and_s.mt')
mt_2 = hl.read_matrix_table(localfs_path+'poles_zerosunion.mt')

FatalError: HailException: No file or directory found at /localfs/4138335/poles_zerosunion.mt

Java stack trace:
is.hail.utils.HailException: No file or directory found at /localfs/4138335/poles_zerosunion.mt
	at is.hail.utils.ErrorHandling.fatal(ErrorHandling.scala:17)
	at is.hail.utils.ErrorHandling.fatal$(ErrorHandling.scala:17)
	at is.hail.utils.package$.fatal(package.scala:78)
	at is.hail.expr.ir.RelationalSpec$.readMetadata(AbstractMatrixTableSpec.scala:33)
	at is.hail.expr.ir.RelationalSpec$.readReferences(AbstractMatrixTableSpec.scala:69)
	at is.hail.variant.ReferenceGenome$.fromHailDataset(ReferenceGenome.scala:560)
	at is.hail.backend.spark.SparkBackend.pyLoadReferencesFromDataset(SparkBackend.scala:618)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAcces

In [13]:
mts = [mt_1, mt_2]

In [14]:
pheno = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/pheno/GTS-coded-corrected-june-2021.csv',
    impute=True,
    delimiter=',',
    quote="\""
)

pheno = pheno.key_by(pheno.ID)

2023-07-28 10:12:44.839 Hail: INFO: Reading table to impute column types
2023-07-28 10:12:46.159 Hail: INFO: Finished type imputation
  Loading field 'ID' as type str (imputed)
  Loading field 'family' as type str (imputed)
  Loading field 'sex' as type str (imputed)
  Loading field 'kinship' as type str (imputed)
  Loading field 'disease' as type str (imputed)
  Loading field 'phenotype' as type str (imputed)
  Loading field 'add_pheno' as type str (imputed)
  Loading field 'heavy_tics' as type str (imputed)
  Loading field 'heavy_tics_familial' as type str (imputed)
  Loading field 'GTS_ASD_group' as type str (imputed)
  Loading field 'nonCTD' as type str (imputed)


In [15]:
for idx, mt in enumerate(mts):
    mt = mt.annotate_cols(
    group = hl.if_else(
            mt.s.contains('B'),
            'local_controls',
            hl.if_else(
                (mt.s.contains('NA') | mt.s.contains('HG')),
                '1kg_controls',
                hl.if_else(
                    mt.s.contains('polish'),
                    'polish_controls',
                    'GTS'
                )
            )
        )
    )
    
    mt = mt.annotate_cols(phenotypes = pheno[mt.s])
    mts[idx] = mt

In [16]:
#sportsmen vs GTS 40
s_vs_gts = mts[0].filter_cols((mts[0].group == 'local_controls') | (mts[0].phenotypes.family == '.'))
s_vs_gts = s_vs_gts.filter_rows(hl.agg.any(s_vs_gts.GT.is_non_ref()))
s_vs_gts = s_vs_gts.checkpoint(localfs_path+'s_vs_gts.mt')    

# polish no zeros vs GTS 40
pw_vs_gts = mts[1].filter_cols((mts[1].group == 'polish_controls') | (mts[1].phenotypes.family == '.'))
pw_vs_gts = pw_vs_gts.filter_rows(hl.agg.any(pw_vs_gts.GT.is_non_ref()))
pw_vs_gts = pw_vs_gts.checkpoint(localfs_path+'pw_vs_gts.mt') 


2023-07-28 10:17:25.191 Hail: INFO: wrote matrix table with 4753663 rows and 142 columns in 4388 partitions to /localfs/4130551/s_vs_gts.mt
2023-07-28 10:36:47.882 Hail: INFO: wrote matrix table with 3035372 rows and 1240 columns in 4389 partitions to /localfs/4130551/pw_vs_gts.mt


In [14]:
s_vs_gts = hl.read_matrix_table(localfs_path+'s_vs_gts.mt')  
pw_vs_gts = hl.read_matrix_table(localfs_path+'pw_vs_gts.mt') 

In [15]:
s_vs_gts = s_vs_gts.naive_coalesce(500)
pw_vs_gts = pw_vs_gts.naive_coalesce(500)

In [5]:
s_vs_gts = s_vs_gts.checkpoint(localfs_path+'s_vs_gts_rep.mt') 

NameError: name 's_vs_gts' is not defined

In [17]:
pw_vs_gts = pw_vs_gts.checkpoint(localfs_path+'pw_vs_gts_rep.mt') 

2023-07-28 16:39:07.547 Hail: INFO: wrote matrix table with 3035372 rows and 1240 columns in 488 partitions to /localfs/4130551/pw_vs_gts_rep.mt


In [4]:
mts = [
    s_vs_gts,
    pw_vs_gts
]
mts_2 = [
    s_vs_gts,
    pw_vs_gts
]
mts_paths = [
    's_vs_gts',
    'pw_vs_gts'
]

NameError: name 's_vs_gts' is not defined

In [ ]:
for idx, mt in enumerate(mts):
    mts[idx] = run_pca(mt, mts_paths[idx], 'three_')

In [ ]:
mts_paths = [
    's_vs_gts',
    'pw_vs_gts'
]

for mt_path in mts_paths:
    mt = hl.read_matrix_table(localfs_path+'after_pca_20_three_'+mt_path)
    mt.write(
        '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/redone_after_pca_20_'+mt_path+'.mt',
        overwrite = True
    )

In [6]:
mts_paths = [
    's_vs_gts',
    'pw_vs_gts'
]

mts = []

for mt_path in mts_paths:
    mt = hl.read_matrix_table(
        '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/redone_after_pca_20_'+mt_path+'.mt'
    )
    
    mts.append(mt)

In [9]:
mts_2 = []

for idx, mt in enumerate(mts):
     mts_2.append(run_pca_no_filter(mt, mts_paths[idx], 'one'))

2023-07-30 06:57:24.333 Hail: INFO: wrote matrix table with 474526 rows and 142 columns in 488 partitions to /localfs/4138335/subset_ones_vs_gts
    Total size: 3.15 GiB
    * Rows/entries: 3.15 GiB
    * Columns: 39.82 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  2425 rows (18.28 MiB)
2023-07-30 06:58:07.695 Hail: INFO: hwe_normalize: found 469968 variants after filtering out monomorphic sites.
2023-07-30 06:58:12.298 Hail: INFO: pca: running PCA with 20 components.../ 488]
2023-07-30 06:58:44.977 Hail: INFO: wrote table with 0 rows in 0 partitions to /localfs/4138335/tmp_hail/persist_table4LCyw0UDZ7
    Total size: 23.24 KiB
    * Rows: 0.00 B
    * Globals: 23.24 KiB
    * Smallest partition: N/A
    * Largest partition:  N/A
2023-07-30 07:00:07.504 Hail: INFO: wrote matrix table with 4753663 rows and 142 columns in 488 partitions to /localfs/4138335/after_pca_no_filters_ones_vs_gts
2023-07-30 07:01:13.752 Hail: INFO: wrote matrix t

In [6]:
mts_paths = [
    's_vs_gts',
    'pw_vs_gts'
]

In [7]:
mts = []

for mt_path in mts_paths:
    mt = hl.read_matrix_table(localfs_path+'after_pca_no_filters_one'+mt_path)
    mts.append(mt)

In [8]:
mts_2 = []

for mt_path in mts_paths:
    
    mt = hl.read_matrix_table(
        '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/redone_after_pca_20_'+mt_path+'.mt'
    ) 
    
    mts_2.append(mt)

In [ ]:
for idx, mt in enumerate(mts):
    
    mt = mt.annotate_cols(pruned_scores = mts_2[idx].cols()[mt.s].pruned_scores) #these are the pruned scores
    
    mt.write(
        '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/'+mts_paths[idx]+'_pca.mt',
        overwrite = True
    ) 

2023-07-30 07:32:53.296 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


In [17]:
mts_paths = [
    's_vs_gts',
    'pw_vs_gts'
]

mts = []

for path in mts_paths:

    mt = hl.read_matrix_table(
        '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/'+path+'_pca.mt'
    )
    
    mts.append(mt)

In [20]:
# remove PCA outliers

mtx_s_2 = []

for idx, mt in enumerate(mts):
    
    mtx = remove_pca_outliers(mt, 'pruned_scores', 2, mts_paths[idx], '_no_outs_pruned_2')
    mtx_2 = remove_pca_outliers(mt, 'scores_no_filter', 2, mts_paths[idx], '_no_outs_subs_2')
    
    mtx = mtx.drop(
        mtx['scores_no_filter'],
        mtx['pruned_scores']
    )
    
    mtx_2 = mtx_2.drop(
        mtx_2['scores_no_filter'],
        mtx_2['pruned_scores']
    )
    
    mts[idx] = mtx 
    mtx_s_2.append(mtx_2)

2023-07-30 08:45:45.318 Hail: INFO: wrote matrix table with 4753663 rows and 140 columns in 488 partitions to /localfs/4138335/no_outlierss_vs_gts_no_outs_pruned_2.mt
2023-07-30 08:47:34.990 Hail: INFO: wrote matrix table with 4753663 rows and 140 columns in 488 partitions to /localfs/4138335/no_outlierss_vs_gts_no_outs_subs_2.mt
2023-07-30 08:49:04.607 Hail: INFO: wrote matrix table with 3035372 rows and 1238 columns in 488 partitions to /localfs/4138335/no_outlierspw_vs_gts_no_outs_pruned_2.mt
2023-07-30 08:50:32.847 Hail: INFO: wrote matrix table with 3035372 rows and 1238 columns in 488 partitions to /localfs/4138335/no_outlierspw_vs_gts_no_outs_subs_2.mt


In [21]:
for mt in mts:
    print(mt.count()) 

(4753663, 140)
(3035372, 1238)


In [22]:
for mtx in mtx_s_2:
    print(mtx.count()) # the method makes no difference, always just two GTS samples are excluded. Which?
    
    

(4753663, 140)
(3035372, 1238)


In [28]:
mt_og = hl.read_matrix_table(localfs_path+'after_pca_no_filters_onepw_vs_gts')

In [30]:
og_samples = mt_og.filter_cols(mt_og.group == 'GTS').s.collect()

In [35]:
to_keep = mts[1].filter_cols(mts[1].group == 'GTS').s.collect()

In [39]:
list(set(og_samples).difference(to_keep)) # thease are two more samples to exclude

['WGS_5', 'WGS_8']